Après avoir sélectionné la méthode de clustering la plus efficace, nous allons analyser le vocabulaire utilisé dans les tweets de chaque cluster.
Avant toute chose, il est nécessaire de nettoyer les tweets, c'est-à-dire de supprimer la ponctuation, les majuscules, les caractères spéciaux, les emojis, etc.
On va également créer une liste de stopwords à supprimer.
La fonction suivante prend en argument un texte et renvoie le texte nettoyé.

In [1]:
!pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.5 MB/s eta 0:00:00


In [3]:
!pip install wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 12.1 MB/s eta 0:00:0000:01


In [4]:
import json
import pandas as pd
import string
import demoji
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

import wordcloud
import numpy as np
import io
import requests
import PIL
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_json("tweets.json", orient = 'index')

In [ ]:
#rajouter les mots qui apparaissent très souvent et qui ne sont pas utiles pour l'analyse
stop_words_context = ["france", "pologne", "but"]
stop_words = set(stopwords.words('french'))
stop_words = list(stop_words) + stop_words_context

def rm_stopwords(text):
    return [w for w in text.split() if w not in stop_words]

In [ ]:
def clean(text):
    #tout mettre en minuscules
    text = text.lower()
    #suppression de la ponctuation
    punct = string.punctuation
    text = text.translate(str.maketrans(", ", punct))
    #suppression des chiffres (pour une analyse de vocabulaire, ils ne sont pas nécessaires)
    text = ''.join([i for i in text if not i.isdigit()])
    #suppression des emojis
    for item in demoji.findall(text):
        text =text.replace(item,"")
    #suppression des mentions
    text = re.sub("@[A-Za-z0-9_]+","", text)
    #suppression des hashtags
    text = re.sub("#[A-Za-z0-9_]+","", text)
    #suppression des stopwords
    text = rm_stopwords(text)
    return text

On va ensuite créer un wordcloud pour chaque cluster.
Première étape : dans le dataframe des tweets, créer une nouvelle variable "clean text" qui contient le texte des tweets modifiés grâce à la fonction clean.

In [ ]:
df["clean_text"] = df["text"].apply(lambda x: clean(x))

In [ ]:
fig = plt.figure()

def make_wordcloud(corpus):
    wc = wordcloud.WordCloud(background_color="white", max_words=2000)
    wc.generate(corpus)
    return wc

#j'ai repris le code du TP, il faudra faire une boucle pour appliquer ça à chaque cluster
plt.imshow(make_wordcloud(dumas), interpolation='bilinear')
plt.axis("off")

On va maintenant créer différentes variables concernant les tweets, de manière à analyser d'autres caractéristiques que le vocabulaire.
Pour notre analyse, il peut être utile de connaître le nombre de hashtags dans chaque tweet, le nombre de majuscules ainsi que le nombre de points d'exclamation.
Le dataframe initial contient déjà le nombre de likes et de retweets.

In [ ]:
def count_hashtags(text):
    find = re.compile("([#]\w+)").findall(text)
    return len(find)

df["nb_hashtags"] = df["text"].apply(lambda x : count_hashtags(x))

def count_exclamation(text):
    find = re.compile("(\w?\s?[!])").findall(text)
    return len(find)

df["nb_exclamation"] = df["text"].apply(lambda x : count_exclamation(x))

def count_maj(text):
    find = re.compile("([A-Z][A-Z]+)").findall(text)
    return len(find)

df["nb_maj"] = df["text"].apply(lambda x : count_maj(x))

Statistiques descriptives sur les variables, par cluster.

In [ ]:
round(df.describe(), 3)

Il faut ajouter une variable "cluster" dans le dataframe, qui contient l'identifiant du cluster auquel appartient chaque utilisateur.